In [42]:
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import pickle
import random
from multiprocessing import Pool

In [2]:

data_root = '/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/'
os.listdir(data_root)

['Astrocytes_sALSvsCTRL',
 'IN-SST_C9ALSvsCTRL',
 'L4_C9ALSvsCTRL',
 'Inhibitory_sALSvsCTRL',
 'L5-6-CC_C9ALSvsCTRL',
 'Excitatory_sALSvsCTRL',
 'L5-6_C9ALSvsCTRL',
 'Excitatory_C9ALSvsCTRL',
 'Astrocytes_C9ALSvsCTRL',
 'L2-3_sALSvsCTRL',
 'AST-FB_sALSvsCTRL',
 'Oligodendrocytes_C9ALSvsCTRL',
 'get_fasta_for_switches.sh',
 'sALS_ALL_training_test_data.pkl',
 'train_data.npy',
 'OPC_C9ALSvsCTRL',
 'all_seqs_celltypes_input',
 'valid_data.npy',
 'REDU_plots',
 'tst_train_data.npy',
 'IN-VIP_C9ALSvsCTRL',
 'L4_sALSvsCTRL',
 'AST-PP_C9ALSvsCTRL',
 'IN-VIP_sALSvsCTRL',
 'AST-PP_sALSvsCTRL',
 'L5-6-CC_sALSvsCTRL',
 'all_seqs_celltypes_input.pkl',
 'IN-PV_C9ALSvsCTRL',
 'c9als_all_seqs_celltypes_input.pkl',
 'Oligodendrocytes_sALSvsCTRL',
 'IN-SST_sALSvsCTRL',
 'OPC_sALSvsCTRL',
 'L2-3_C9ALSvsCTRL',
 'test_data.npy',
 'Microglia_sALSvsCTRL',
 'Inhibitory_C9ALSvsCTRL',
 'C9ALS_ALL_training_test_data.pkl',
 'Microglia_C9ALSvsCTRL',
 'Endothelial_ALSvsCTRL',
 'TF_modisco',
 'IN-PV_sALSvsCTRL',
 

In [79]:
# C9ALS first # keep main celltyeps only
ct = ['Excitatory_C9ALSvsCTRL', 'Astrocytes_C9ALSvsCTRL','Oligodendrocytes_C9ALSvsCTRL',
'OPC_C9ALSvsCTRL', 'Inhibitory_C9ALSvsCTRL', 'Microglia_C9ALSvsCTRL']

In [6]:
sequences_dict = {}
for ct_cn in ct:
    inp_fa = data_root + "/{}/switch_DNA_sequence.fa".format(ct_cn)
    inp_fa = SeqIO.parse(inp_fa, "fasta")
    for rec in inp_fa:
        if rec.id not in sequences_dict:
            sequences_dict[rec.id] = str(rec.seq)
        else:
            continue 
print(len(sequences_dict))


103275


In [81]:
celltypes = [e.split('_')[0] for e in ct]
celltypes = sorted(celltypes)

In [36]:
def transcribe_positive_strand(seq):
    """ input is the 5' to 3' coding squence
        so the RNA will be exact sequence except
        U instead of T
    """
    return(seq.replace('T','U'))

def transcribe_negative_strand(seq):
    """ input is the 5' to 3' template squence
        so the function complement and returns
        the reverse of sequence
    """
    complement = {'A': 'U', 'C': 'G', 'G': 'C', 'T': 'A'}
    return "".join(complement.get(base, base) for base in reversed(seq))

In [37]:
transcribed_sequences = {}
for key,value in sequences_dict.items():
    strand = key.split(':')[-1]
    if strand == '+':
        transcribed_sequences[key] = transcribe_positive_strand(value)
    else:
        transcribed_sequences[key] = transcribe_negative_strand(value)

In [38]:
len(transcribed_sequences)

103275

In [40]:
def get_1h_seq(seq):
    """
    This is a simple code to get one-hot representaton of the
    senquences.
    --------------
    Arguments:
    seq: RNA sequence in fasta format
    """
    nt_code = {
        "N": [0, 0, 0, 0],
        "A": [1, 0, 0, 0],
        "C": [0, 1, 0, 0],
        "G": [0, 0, 1, 0],
        "U": [0, 0, 0, 1],
    }

    seq_1h = [nt_code[nt] for nt in seq]
    return np.array(seq_1h)

def makes_seqs_ready(seq,max_len,left_pad_max):
    """
    takes in the sequence, max len and max left pad
    and returns the one hot representation of sequence with maximum pad
    """
    if len(seq) < max_len:
        diff = max_len - len(seq)
        lp = random.randint(0, left_pad_max)
        lp = min(lp, diff)
        seq = get_1h_seq(
            "N" * (lp) + str(seq) + "N" * (max_len - len(seq) - lp)
        )
    else:
        seq = get_1h_seq(str(seq))
    seq = np.array(seq, dtype=np.int8).swapaxes(0, 1)
    
    return seq


In [43]:
one_hot_values = { key : makes_seqs_ready(seq, 16000, 10) for key, seq in transcribed_sequences.items()}

In [47]:
keys = list(one_hot_values.keys())
seqs = [transcribed_sequences[k] for k in keys]
indexes = list(range(len(keys)))
onehots = [one_hot_values[k] for k in keys]

seqs_data = pd.DataFrame([indexes, keys, seqs, onehots]).T
seqs_data.columns = ['switch_index', 'switch_id', 'sequence', 'one_hot']
seqs_data

,switch_index,switch_id,sequence,one_hot
0,0,chr12:AACS:125140928:125143316:+,GUGAGGCGGGACAAACUUGUCUUCCUCACACCCAUCUUACUUCCUC...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,..."
1,1,chr15:AAGAB:67201028:67201672:-,GGAGGUUAAGGAGAAAUCUUUUUUUUCCUCAGUAUAUUGUAAGAGA...,"[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,..."
2,2,chr15:AAGAB:67201028:67202710:-,ACUACAUCAUAAACAUGUCUUUGAAACCCGUCUCCCAUCUUCUAGU...,"[[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,..."
3,3,chr15:AAGAB:67201672:67202710:-,ACUACAUCAUAAACAUGUCUUUGAAACCCGUCUCCCAUCUUCUAGU...,"[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,..."
4,4,chr15:AAGAB:67202710:67217079:-,AUCUAUUAUAGUUCAAUCCCCAGUAAUGCAGAUGGAGGAGGAUUAG...,"[[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,..."
...,...,...,...,...
103270,103270,chr1:ZZZ3:77626869:77627900:-,GAAACGUUCUCCAGGUAAUUGCUUUUUGAAACCGAGGUUGAGCAUU...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,..."
103271,103271,chr1:ZZZ3:77626869:77631145:-,CAAUUUGAGGAGAAUAAUCUUAGUCCUAAUGAAACAAAUGCAACUG...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,..."
103272,103272,chr1:ZZZ3:77627900:77629840:-,AGAUCAAAGUAUCAAAUCCUGGUUGUGGGACAUAAUAUUUUUUUUG...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,..."
103273,103273,chr1:ZZZ3:77627900:77631145:-,CAAUUUGAGGAGAAUAAUCUUAGUCCUAAUGAAACAAAUGCAACUG...,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,..."


In [48]:
seqs_data.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/All_sequences.csv')

In [82]:
import scipy.stats as ss

def rank_to_normal(rank, c, n):
    # Standard quantile function
    x = (rank - c) / (n - 2*c + 1)
    return ss.norm.ppf(x)

def rank_INT(series, c=3.0/8, stochastic=True):
    """ Perform rank-based inverse normal transformation on pandas series.
        If stochastic is True ties are given rank randomly, otherwise ties will
        share the same value. NaN values are ignored.
        Args:
            param1 (pandas.Series):   Series of values to transform
            param2 (Optional[float]): Constand parameter (Bloms constant)
            param3 (Optional[bool]):  Whether to randomise rank of ties
        
        Returns:
            pandas.Series
    """

    # Check input
    #assert(isinstance(series, pd.Series))
    #assert(isinstance(c, float))
    #assert(isinstance(stochastic, bool))

    # Set seed
    np.random.seed(123)

    # Take original series indexes
    orig_idx = series.index

    # Drop NaNs
    series = series.loc[~pd.isnull(series)]

    # Get ranks
    if stochastic == True:
        # Shuffle by index
        series = series.loc[np.random.permutation(series.index)]
        # Get rank, ties are determined by their position in the series (hence
        # why we randomised the series)
        rank = ss.rankdata(series, method="ordinal")
    else:
        # Get rank, ties are averaged
        rank = ss.rankdata(series, method="average")

    # Convert numpy array back to series
    rank = pd.Series(rank, index=series.index)

    # Convert rank to normal distribution
    transformed = rank.apply(rank_to_normal, c=c, n=len(rank))
    
    return transformed[orig_idx]

def get_sig_lfc(df, name):
    tmp_df =  df.loc[df['switch_name'] == name]
    lfc = round(float(tmp_df['LFC_rand_INT']),4)
    return(lfc)
    
def get_sig_mult(val):
    if val > 1.3:
        return 1
    else:
        return 0
## this will take couple of minutes ~ 10-15 min 
labels = {}
for key in sequences_dict.keys():
    labels[key] = {}
for ct in celltypes:
    print(ct)
    df_name = data_root + ct + "_C9ALSvsCTRL/APAlog_res_metadata_added.tsv"
    inp_df = pd.read_csv(df_name, sep='\t')
    ## get significant APA switches
    inp_df['sig_multiplyer'] = inp_df['negative_logFDR'].apply(get_sig_mult) 
    inp_df['sig_LFC_PA_Usage'] = inp_df['sig_multiplyer'] * inp_df['LFC_PA_Usage']
    inp_df = inp_df[~pd.isnull(inp_df['sig_LFC_PA_Usage'])] ## remove NaNs
    inp_df['LFC_rand_INT'] = rank_INT(inp_df['sig_LFC_PA_Usage']) 
    for i in range(inp_df.shape[0]):
        try:
            seq_id = inp_df.iloc[i]['switch_name']
            labels[seq_id][ct] = get_sig_lfc(inp_df, seq_id)
        except:
            continue

Astrocytes
Excitatory
Inhibitory
Microglia
OPC
Oligodendrocytes


In [85]:
# get the pd dataframe from labels
seq_ids = list(labels.keys())
seq_ids_all = []
celltypes_ids = []
PA_vals = []
for seq_id in seq_ids:
    for ct in labels[seq_id].keys():
        celltypes_ids.append(ct)
        PA_vals.append(labels[seq_id][ct])
        seq_ids_all.append(seq_id)
print(len(seq_ids), len(celltypes_ids), len(PA_vals))


103275 221862 221862


In [88]:
print(len(seq_ids_all), len(celltypes_ids), len(PA_vals))

221862 221862 221862


In [91]:
seq_id_idx = dict(zip(seqs_data['switch_id'], seqs_data['switch_index']))
seq_id_idx

{'chr12:AACS:125140928:125143316:+': 0,
 'chr15:AAGAB:67201028:67201672:-': 1,
 'chr15:AAGAB:67201028:67202710:-': 2,
 'chr15:AAGAB:67201672:67202710:-': 3,
 'chr15:AAGAB:67202710:67217079:-': 4,
 'chr2:AAK1:69457997:69461526:-': 5,
 'chr2:AAK1:69457997:69464011:-': 6,
 'chr2:AAK1:69457997:69467864:-': 7,
 'chr2:AAK1:69457997:69470919:-': 8,
 'chr2:AAK1:69461526:69464011:-': 9,
 'chr2:AAK1:69461526:69467864:-': 10,
 'chr2:AAK1:69461526:69470919:-': 11,
 'chr2:AAK1:69461526:69474509:-': 12,
 'chr2:AAK1:69464011:69467864:-': 13,
 'chr2:AAK1:69464011:69470919:-': 14,
 'chr2:AAK1:69464011:69474509:-': 15,
 'chr2:AAK1:69467864:69470919:-': 16,
 'chr2:AAK1:69467864:69474509:-': 17,
 'chr2:AAK1:69470919:69474509:-': 18,
 'chr16:AARS:70252298:70262346:-': 19,
 'chr16:AARS:70252417:70262346:-': 20,
 'chr16:AARS:70262346:70276519:-': 21,
 'chr6:AARS2:44298731:44299560:-': 22,
 'chr6:AARS2:44298731:44300312:-': 23,
 'chr6:AARS2:44299560:44300312:-': 24,
 'chr11:AASDHPPT:106091478:106097048:+': 25

In [92]:
seq_idx = [seq_id_idx[seq_id] for seq_id in seq_ids_all]

In [93]:
label_data = pd.DataFrame([seq_ids_all,seq_idx,celltypes_ids, PA_vals]).T
label_data.columns = ['switch_id','switch_inx','celltype','APA_lfc']
label_data

,switch_id,switch_inx,celltype,APA_lfc
0,chr12:AACS:125140928:125143316:+,0,Astrocytes,-0.2384
1,chr12:AACS:125140928:125143316:+,0,Excitatory,-1.2717
2,chr12:AACS:125140928:125143316:+,0,Inhibitory,0.3739
3,chr12:AACS:125140928:125143316:+,0,OPC,0.4887
4,chr12:AACS:125140928:125143316:+,0,Oligodendrocytes,1.0915
...,...,...,...,...
221857,chr1:ZZZ3:77626869:77627900:-,103270,Microglia,0.696
221858,chr1:ZZZ3:77626869:77631145:-,103271,Microglia,0.303
221859,chr1:ZZZ3:77627900:77629840:-,103272,Microglia,-1.159
221860,chr1:ZZZ3:77627900:77631145:-,103273,Microglia,-0.0584


In [94]:
label_data.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/All_labels.csv')

In [97]:
def split_data(df):
    """
    takes in the pandas dataframe and split to train, test and valid datasets
    """
    df = df[df['APA_lfc'].notna()]
    test_index = np.array(random.sample(range(df.shape[0]), int(float(df.shape[0])*0.15)))
    mask = np.zeros(df.shape[0],dtype=bool)
    mask[test_index] = True
    df_test = df[mask]
    df_test.head()
    tmp_df = df[~mask]
    valid_index = np.array(random.sample(range(tmp_df.shape[0]), int(float(tmp_df.shape[0])*0.05)))
    mask = np.zeros(tmp_df.shape[0],dtype=bool)
    mask[valid_index] = True
    df_valid = tmp_df[mask]
    df_train = tmp_df[~mask]
    
    return (df_train, df_valid, df_test)

df_train, df_valid, df_test = split_data(label_data)
print(df_train.shape, df_valid.shape, df_test.shape)
df_train.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/train_data.csv')
df_valid.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/valid_data.csv')
df_test.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/test_data.csv')

(179154, 4) (9429, 4) (33279, 4)


# lets repeat everything for sALS data and call it donw

In [99]:
# C9ALS first # keep main celltyeps only
ct = ['Excitatory_sALSvsCTRL', 'Astrocytes_sALSvsCTRL','Oligodendrocytes_sALSvsCTRL',
'OPC_sALSvsCTRL', 'Inhibitory_sALSvsCTRL', 'Microglia_sALSvsCTRL']

celltypes = [e.split('_')[0] for e in ct]
celltypes = sorted(celltypes)

sequences_dict = {}
for ct_cn in ct:
    inp_fa = data_root + "/{}/switch_DNA_sequence.fa".format(ct_cn)
    inp_fa = SeqIO.parse(inp_fa, "fasta")
    for rec in inp_fa:
        if rec.id not in sequences_dict:
            sequences_dict[rec.id] = str(rec.seq)
        else:
            continue 
print(len(sequences_dict))

transcribed_sequences = {}
for key,value in sequences_dict.items():
    strand = key.split(':')[-1]
    if strand == '+':
        transcribed_sequences[key] = transcribe_positive_strand(value)
    else:
        transcribed_sequences[key] = transcribe_negative_strand(value)



one_hot_values = { key : makes_seqs_ready(seq, 16000, 10) for key, seq in transcribed_sequences.items()}
keys = list(one_hot_values.keys())
seqs = [transcribed_sequences[k] for k in keys]
indexes = list(range(len(keys)))
onehots = [one_hot_values[k] for k in keys]

seqs_data = pd.DataFrame([indexes, keys, seqs, onehots]).T
seqs_data.columns = ['switch_index', 'switch_id', 'sequence', 'one_hot']
print(seqs_data.shape)
seqs_data.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/sALS/All_sequences.csv')


## this will take couple of minutes ~ 10-15 min 
labels = {}
for key in sequences_dict.keys():
    labels[key] = {}
for ct in celltypes:
    print(ct)
    df_name = data_root + ct + "_C9ALSvsCTRL/APAlog_res_metadata_added.tsv"
    inp_df = pd.read_csv(df_name, sep='\t')
    ## get significant APA switches
    inp_df['sig_multiplyer'] = inp_df['negative_logFDR'].apply(get_sig_mult) 
    inp_df['sig_LFC_PA_Usage'] = inp_df['sig_multiplyer'] * inp_df['LFC_PA_Usage']
    inp_df = inp_df[~pd.isnull(inp_df['sig_LFC_PA_Usage'])] ## remove NaNs
    inp_df['LFC_rand_INT'] = rank_INT(inp_df['sig_LFC_PA_Usage']) 
    for i in range(inp_df.shape[0]):
        try:
            seq_id = inp_df.iloc[i]['switch_name']
            labels[seq_id][ct] = get_sig_lfc(inp_df, seq_id)
        except:
            continue

# get the pd dataframe from labels
seq_ids = list(labels.keys())
seq_ids_all = []
celltypes_ids = []
PA_vals = []
for seq_id in seq_ids:
    for ct in labels[seq_id].keys():
        celltypes_ids.append(ct)
        PA_vals.append(labels[seq_id][ct])
        seq_ids_all.append(seq_id)
print(len(seq_ids), len(celltypes_ids), len(PA_vals))
label_data.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/sALS/All_labels.csv')

df_train, df_valid, df_test = split_data(label_data)
print(df_train.shape, df_valid.shape, df_test.shape)
df_train.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/sALS/train_data.csv')
df_valid.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/sALS/valid_data.csv')
df_test.to_csv('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/data_for_DL/sALS/test_data.csv')

101993
